# Efficient Yelp API Calls by Wesley Giles

## Import necessary modules and environment variables

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
from dotenv import load_dotenv
load_dotenv()

True

## Now to create the Yelp Client

In [3]:
yelp = YelpAPI(os.environ["YELP_API_KEY"],timeout_s=5.0)
yelp

## Make the file path for the json file

In [14]:
def add_results(file_name,location,term):
  try:
    os.makedirs("./data/")
  except:
    pass
  path = f"./data/{file_name}"
  try:
    with open(path,"r") as f:
      results = json.load(f)
  except:
    print("Initializing new file as empty list")
    with open(path,"w") as f:
      f.writelines(["[]"])
      results = []
  new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  page_amount = len(new_results["businesses"])
  num_pages = math.ceil((new_results["total"] - len(results)) / page_amount)
  for i in tqdm_notebook(range(num_pages)):
    time.sleep(0.2)
    with open(path,"w") as f:
      results.extend(new_results["businesses"])
      json.dump(results,f)
      new_results = yelp.search_query(location=location,
                                  term=term,
                                  offset=len(results))
  return results

In [19]:
len(add_results("results.json","43017","Restauraunt"))

Initializing new file as empty list


  0%|          | 0/1 [00:00<?, ?it/s]

14

In [20]:
len(add_results("results.json","43017","Fast Food"))


  0%|          | 0/29 [00:00<?, ?it/s]

577

## Now to export it to a dataframe

In [21]:
df = pd.read_json("./data/results.json")
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,n0elRo8l89L__9iqyFXJWw,the-pearl-dublin,The Pearl,https://s3-media4.fl.yelpcdn.com/bphoto/fSzRZO...,False,https://www.yelp.com/biz/the-pearl-dublin?adju...,166,"[{'alias': 'newamerican', 'title': 'American (...",4.5,"{'latitude': 40.10152, 'longitude': -83.113805}",[],"{'address1': '88 N High St', 'address2': None,...",+16146956255,(614) 695-6255,2395.191348,NaN
1,tK_pr-xqHiWwdN3wrsW4aA,the-rail-dublin-dublin,The Rail - Dublin,https://s3-media1.fl.yelpcdn.com/bphoto/R5BXuf...,False,https://www.yelp.com/biz/the-rail-dublin-dubli...,537,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,"{'latitude': 40.07318, 'longitude': -83.122015}","[delivery, pickup]","{'address1': '5839 Frantz Rd', 'address2': Non...",+16147253200,(614) 725-3200,5388.739953,$$
2,JCpnFNhJR11jTKERdRL5hw,bakersfield-columbus,Bakersfield,https://s3-media1.fl.yelpcdn.com/bphoto/jFM7CG...,False,https://www.yelp.com/biz/bakersfield-columbus?...,735,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 39.97733, 'longitude': -83.00374}",[delivery],"{'address1': '733 N High St', 'address2': '', ...",+16147548436,(614) 754-8436,19024.971809,$$
3,hSNMWcc63YvTWLf22pt5xA,kitchen-social-columbus,Kitchen Social,https://s3-media1.fl.yelpcdn.com/bphoto/AD95dG...,False,https://www.yelp.com/biz/kitchen-social-columb...,610,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 40.14759, 'longitude': -82.97463}",[delivery],"{'address1': '8954 Lyra Dr', 'address2': '', '...",+16145058492,(614) 505-8492,13032.365346,$$
4,NprP2naEmVi9-KpDTNr2gA,benihana-columbus,Benihana,https://s3-media3.fl.yelpcdn.com/bphoto/X9am0t...,False,https://www.yelp.com/biz/benihana-columbus?adj...,310,"[{'alias': 'japanese', 'title': 'Japanese'}]",2.5,"{'latitude': 40.1446152, 'longitude': -82.9758...","[delivery, pickup]","{'address1': '8781 Lyra Dr', 'address2': '', '...",+16144363705,(614) 436-3705,12726.178259,$$


In [23]:
df.duplicated(subset=["id"]).sum()

9

In [25]:
df.drop_duplicates(subset=["id"], inplace = True)
df.duplicated(subset=["id"]).sum()

0

In [26]:
df.to_csv("./data/results.csv.gz", compression="gzip", index = False)